# 第十二章 Context

## 章節概述

Context 是 Go 1.7 引入的重要套件，提供了跨 API 邊界和 goroutine 之間傳遞請求範圍值、取消信號和截止時間的標準方式。Context 在現代 Go 應用程式中扮演著核心角色，特別是在 Web 服務、資料庫操作和 gRPC 通訊中。

### 本章重點內容
- **Context 的基本概念**: 理解 Context 的用途和重要性
- **取消機制**: 如何優雅地取消長時間運行的操作
- **超時控制**: 設定和管理操作的時間限制
- **值傳遞**: 在請求鏈中安全地傳遞值
- **最佳實踐**: Context 的正確使用方式和常見陷阱

### Context 的核心理念
Context 的設計基於以下原則：
1. **不可變性**: Context 一旦建立就不能修改
2. **樹狀結構**: 子 Context 繼承父 Context 的屬性
3. **取消傳播**: 父 Context 取消會導致所有子 Context 取消
4. **線程安全**: 可以在多個 goroutine 中安全使用

In [ ]:
// Context 綜合示範
package main

import (
    "context"
    "fmt"
    "math/rand"
    "net/http"
    "sync"
    "time"
)

func main() {
    fmt.Println("=== Context 綜合示範 ===")
    
    // 1. 基本 Context 概念
    fmt.Println("\n1. 基本 Context:")
    demonstrateBasicContext()
    
    // 2. 取消機制
    fmt.Println("\n2. 取消機制:")
    demonstrateCancellation()
    
    // 3. 超時控制
    fmt.Println("\n3. 超時控制:")
    demonstrateTimeout()
    
    // 4. 值傳遞
    fmt.Println("\n4. 值傳遞:")
    demonstrateValuePassing()
    
    // 5. 實際應用場景
    fmt.Println("\n5. 實際應用 - HTTP 服務:")
    demonstrateRealWorldUsage()
}

// 示範基本 Context 概念
func demonstrateBasicContext() {
    // 1. 背景 Context - 通常用作根 Context
    bgCtx := context.Background()
    fmt.Printf("  背景 Context: %T\n", bgCtx)
    
    // 2. TODO Context - 當不確定使用什麼 Context 時的佔位符
    todoCtx := context.TODO()
    fmt.Printf("  TODO Context: %T\n", todoCtx)
    
    // 3. 檢查 Context 狀態
    select {
    case <-bgCtx.Done():
        fmt.Println("  背景 Context 已取消")
    default:
        fmt.Println("  背景 Context 正常運行")
    }
    
    fmt.Printf("  錯誤狀態: %v\n", bgCtx.Err())
    fmt.Printf("  截止時間: %v\n", bgCtx.Deadline())
}

// 示範取消機制
func demonstrateCancellation() {
    // 建立可取消的 Context
    ctx, cancel := context.WithCancel(context.Background())
    defer cancel() // 確保資源清理
    
    var wg sync.WaitGroup
    
    // 啟動工作 goroutine
    wg.Add(1)
    go func() {
        defer wg.Done()
        doWork(ctx, "工作者1", 3*time.Second)
    }()
    
    wg.Add(1)
    go func() {
        defer wg.Done()
        doWork(ctx, "工作者2", 5*time.Second)
    }()
    
    // 讓工作執行一段時間
    time.Sleep(2 * time.Second)
    
    fmt.Println("  發送取消信號...")
    cancel() // 取消所有工作
    
    // 等待所有工作完成
    wg.Wait()
    fmt.Println("  所有工作已停止")
}

// 模擬可取消的工作
func doWork(ctx context.Context, name string, duration time.Duration) {
    fmt.Printf("  %s 開始工作，預計耗時 %v\n", name, duration)
    
    ticker := time.NewTicker(500 * time.Millisecond)
    defer ticker.Stop()
    
    deadline := time.Now().Add(duration)
    
    for {
        select {
        case <-ctx.Done():
            fmt.Printf("  %s 收到取消信號: %v\n", name, ctx.Err())
            return
        case <-ticker.C:
            if time.Now().After(deadline) {
                fmt.Printf("  %s 正常完成工作\n", name)
                return
            }
            fmt.Printf("  %s 工作進行中...\n", name)
        }
    }
}

// 示範超時控制
func demonstrateTimeout() {
    // 1. WithTimeout - 設定固定超時時間
    fmt.Println("  測試 WithTimeout:")
    ctx1, cancel1 := context.WithTimeout(context.Background(), 2*time.Second)
    defer cancel1()
    
    start := time.Now()
    err := simulateNetworkCall(ctx1, "API調用1", 3*time.Second)
    if err != nil {
        fmt.Printf("  操作失敗: %v (耗時: %v)\n", err, time.Since(start))
    }
    
    // 2. WithDeadline - 設定絕對截止時間
    fmt.Println("\n  測試 WithDeadline:")
    deadline := time.Now().Add(1 * time.Second)
    ctx2, cancel2 := context.WithDeadline(context.Background(), deadline)
    defer cancel2()
    
    start = time.Now()
    err = simulateNetworkCall(ctx2, "API調用2", 2*time.Second)
    if err != nil {
        fmt.Printf("  操作失敗: %v (耗時: %v)\n", err, time.Since(start))
    }
    
    // 3. 成功的調用
    fmt.Println("\n  測試成功調用:")
    ctx3, cancel3 := context.WithTimeout(context.Background(), 2*time.Second)
    defer cancel3()
    
    start = time.Now()
    err = simulateNetworkCall(ctx3, "API調用3", 1*time.Second)
    if err != nil {
        fmt.Printf("  操作失敗: %v\n", err)
    } else {
        fmt.Printf("  操作成功 (耗時: %v)\n", time.Since(start))
    }
}

// 模擬網路調用
func simulateNetworkCall(ctx context.Context, name string, duration time.Duration) error {
    fmt.Printf("    %s 開始，預計耗時 %v\n", name, duration)
    
    select {
    case <-time.After(duration):
        fmt.Printf("    %s 完成\n", name)
        return nil
    case <-ctx.Done():
        return ctx.Err()
    }
}

// 自訂鍵類型，避免鍵衝突
type contextKey string

const (
    userIDKey    contextKey = "userID"
    requestIDKey contextKey = "requestID"
    traceIDKey   contextKey = "traceID"
)

// 示範值傳遞
func demonstrateValuePassing() {
    // 建立帶有值的 Context
    ctx := context.Background()
    
    // 添加用戶ID
    ctx = context.WithValue(ctx, userIDKey, "user123")
    
    // 添加請求ID
    ctx = context.WithValue(ctx, requestIDKey, "req-456")
    
    // 添加追蹤ID
    ctx = context.WithValue(ctx, traceIDKey, "trace-789")
    
    fmt.Println("  Context 中的值:")
    printContextValues(ctx)
    
    // 模擬請求處理鏈
    fmt.Println("\n  模擬請求處理鏈:")
    handleRequest(ctx)
}

// 打印 Context 中的值
func printContextValues(ctx context.Context) {
    if userID := ctx.Value(userIDKey); userID != nil {
        fmt.Printf("    用戶ID: %v\n", userID)
    }
    
    if requestID := ctx.Value(requestIDKey); requestID != nil {
        fmt.Printf("    請求ID: %v\n", requestID)
    }
    
    if traceID := ctx.Value(traceIDKey); traceID != nil {
        fmt.Printf("    追蹤ID: %v\n", traceID)
    }
}

// 模擬請求處理
func handleRequest(ctx context.Context) {
    fmt.Println("    進入 handleRequest")
    
    // 模擬認證檢查
    if err := authenticate(ctx); err != nil {
        fmt.Printf("    認證失敗: %v\n", err)
        return
    }
    
    // 模擬業務邏輯處理
    processBusinessLogic(ctx)
}

// 模擬認證
func authenticate(ctx context.Context) error {
    userID := ctx.Value(userIDKey)
    if userID == nil {
        return fmt.Errorf("缺少用戶ID")
    }
    
    fmt.Printf("    認證用戶: %v\n", userID)
    return nil
}

// 模擬業務邏輯
func processBusinessLogic(ctx context.Context) {
    requestID := ctx.Value(requestIDKey)
    traceID := ctx.Value(traceIDKey)
    
    fmt.Printf("    處理業務邏輯 [請求: %v, 追蹤: %v]\n", requestID, traceID)
    
    // 模擬資料庫查詢
    queryDatabase(ctx)
}

// 模擬資料庫查詢
func queryDatabase(ctx context.Context) {
    traceID := ctx.Value(traceIDKey)
    fmt.Printf("    執行資料庫查詢 [追蹤: %v]\n", traceID)
}

// 示範實際應用場景
func demonstrateRealWorldUsage() {
    // 模擬 HTTP 請求處理
    fmt.Println("  模擬 HTTP 請求處理...")
    
    // 建立根 Context
    ctx := context.Background()
    
    // 添加請求資訊
    ctx = context.WithValue(ctx, requestIDKey, generateRequestID())
    ctx = context.WithValue(ctx, userIDKey, "user789")
    
    // 設定請求超時
    ctx, cancel := context.WithTimeout(ctx, 5*time.Second)
    defer cancel()
    
    // 模擬併發處理多個服務調用
    var wg sync.WaitGroup
    
    services := []string{"用戶服務", "訂單服務", "支付服務"}
    
    for _, service := range services {
        wg.Add(1)
        go func(serviceName string) {
            defer wg.Done()
            callService(ctx, serviceName)
        }(service)
    }
    
    wg.Wait()
    fmt.Println("  所有服務調用完成")
}

// 生成請求ID
func generateRequestID() string {
    return fmt.Sprintf("req-%d", rand.Intn(10000))
}

// 模擬服務調用
func callService(ctx context.Context, serviceName string) {
    requestID := ctx.Value(requestIDKey)
    userID := ctx.Value(userIDKey)
    
    fmt.Printf("    調用 %s [請求: %v, 用戶: %v]\n", serviceName, requestID, userID)
    
    // 模擬服務調用時間
    callDuration := time.Duration(rand.Intn(2000)) * time.Millisecond
    
    select {
    case <-time.After(callDuration):
        fmt.Printf("    %s 調用成功 (耗時: %v)\n", serviceName, callDuration)
    case <-ctx.Done():
        fmt.Printf("    %s 調用被取消: %v\n", serviceName, ctx.Err())
    }
}